<a href="https://colab.research.google.com/github/RomaZhm/ColabNotes/blob/main/dz12_kmss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math

def ksi_mod(): # функция моделирования "Кси" (интервалы между приходами требований)
    gamma = np.random.random() # моделирование гамма
    ksi = - math.log(gamma) / 0.5
    return ksi

In [ ]:
def eta_mod(a, b): # функция моделирования "Эта" на [a,b] (время обслуживания на (1,2)ом канале)
    gamma = np.random.random()
    eta = (b - a) * gamma + a
    return eta

In [ ]:
def estimate_f(T):
    T0 = 0
    Tk = [0, 0]
    Ind = 1 # Ind = 1, то есть пока не было потеряно более одного требования
    count = 0 # подсчет потерянных требований
    while T0 <= T: # пока находимся в интервале [0, 150]
      ksi = ksi_mod()
      T0 = T0 + ksi
      if T0 > min(Tk): # есть минимум один свободный канал; ищем первый из свободных
          if Tk[0] < T0: # свободен 1ый канал
              eta = eta_mod(a = 0, b = 2) # моделируем время обслуживания на нем(распределено равномерно на [0,2])
              Tk[0] = T0 + eta
          else: # свободен 2ой
              eta = eta_mod(a = 1, b = 3) # время обслуживания на нем распределено равномерно на [1,3]
              Tk[1] = T0 + eta 
      else: # потеряли требование
          count = count + 1 # записываем потерю
          if count > 1: # не выполнено требование "потеряно требований <= 1":возвращаем 0
              Ind = 0
              return Ind 
          else: # требование выполнено, продолжаем моделировать приходы требований
              continue
    # если за весь интервал [0, 150] не было потеряно более одного требования возвращаем Ind = 1
    return Ind

In [ ]:
def estNmin(teta, delt = 0.1):
    # для дов. вер-ти 92% квантиль равен 1-(1-0.92)/2 = 0.96 => по таблице x_0 = 1.75
    x_0 = 1.75 
    return (x_0**2) * teta / (delt**2)

In [ ]:
def estP(): # оценка вероятности того, что не будет потеряно более одного требования
    N = 500 # выбираем начальное количество моделирований 
    i = 0
    sum = 0 # сумма моделируемых величин
    sum_sq = 0 # сумма квадратов мод. величин
    while True:
        while i < N:
            x = estimate_f(T = 150)
            sum = sum + x
            sum_sq = sum_sq + x**2
            i = i + 1
          
        sum_mean = sum/N # нашли среднее значение суммы
        teta = (sum_sq/N)/(sum_mean**2) - 1 # посчитали тета
        Nmin = math.ceil(estNmin(teta, delt = 0.1)) # посчитали Nmin
        if Nmin <= N:
            return sum_mean, N
        else:
            N = Nmin

In [ ]:
n = 5
print('Несколько оценок: ')
for j in range(n):  
  P = estP()
  print('Вероятность не потерять больше одного требования = ', P[0], ' (Nmin для оценки: ', P[1], ')', sep = '')

Несколько оценок: 
Вероятность не потерять больше одного требования = 0.008452929558920342 (Nmin для оценки: 37975)
Вероятность не потерять больше одного требования = 0.007345024762542766 (Nmin для оценки: 41797)
Вероятность не потерять больше одного требования = 0.00845484282308588 (Nmin для оценки: 36074)
Вероятность не потерять больше одного требования = 0.008377049630585845 (Nmin для оценки: 36409)
Вероятность не потерять больше одного требования = 0.008108393640061275 (Nmin для оценки: 37862)
